Import the Jupyter / PlantGL module

In [69]:
from pgljupyter import *

Tomato model from Fred

In [71]:
%%lpy -w 10 -a True
from random import *

module Support
module Apex(tts)
module DormantBud
module Inflorescence

phyllochrone = 20
daytemp = None

def StartEach():
  global daytemp
  daytemp = 5

Axiom: Support ,(2) _(0.05) Apex(0, 1)

derivation length: 50
production:

Apex(tts, state):
  global daytemp, phyllochrone
  if tts >= phyllochrone:
    nproduce [ &(60) Leaf(0) ] 
    if state == 3:
       nproduce [&(30) Inflorescence(0) ]
    else:
       if random() < 0.5:
          nproduce [&(30) DormantBud(0, randint(20,30)) ]
    state = state % 3
    state +=1
    tts -= phyllochrone
  nproduce /(144) Segment()
  produce Apex(tts+daytemp, state)


DormantBud(tts, dtype):
  tts+=daytemp
  if tts >= dtype:
    nproduce Apex(0,1)
  else:
    produce DormantBud(tts, dtype)

interpretation:
Segment() --> F(0.1)
Leaf(tts) --> ~l(0.5)
Inflorescence(tts) :
  for i in range(5):
    nproduce @Tp(0,0,-1) @Ts(0.2) _(0.02) F(0.2)[+(90)F(0.2) ,(3) @O(0.1)][-(90)F(0.2) ,(3) @O(0.1)]

#Support :
#  nproduce  [@M(0,1)  &(90) 
#  for i in range(20):
#    nproduce +(30) &(-5) F(0.5) @v
#  nproduce ]

endlsystem


LsystemWidget(animate=True, derivationLength=51, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe5…

Have a go at making a simple leaf with logistic growth

In [63]:
%%lpy -w 10 -a True

from math import exp

#this is a definition of the logistic growth function
def growth_logistic(t, L, tip, b):
    return L / (1+exp(-(t-tip)/b))

#phyllochrone is an arbitrary measure of time
phyllochrone = 1
#Li is involved in Internode
Li = 0.5
#Lf is involved in Leaf
Lf = 1
#dt is a timestep
dt = 0.5

#tip is a function of time and leaf
tip = phyllochrone/2
#bi
b_i = 0.15
#bf 
b_f = 0.3

#three components, Apex, Leaf and Internode
Axiom:  I(0) L(0) A(0)
derivation length: 20
production:
A(t): 
    if t > phyllochrone : produce  L(0) /(180) L(0)/(180) I(0) A(0)
    else: produce A(t+dt)

I(t) --> I(t+dt)
L(t) --> L(t+dt)

interpretation:
I(t) --> F(growth_logistic(t, Li, tip, b_i))
L(t) --> [&(90*min(1,t)),(2) ~l(growth_logistic(t, Lf, tip, b_f))]

LsystemWidget(animate=True, derivationLength=21, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe5…

Compound tomato leaf

In [65]:
%%lpy -w 10 -a True

import random 

from numpy.random import seed
from numpy.random import rand
seed(1)

module Leaf
module MinLeaf

leafscales = [0.5,1.5,1,0.5]
Axiom: ,(2) _(0.05) Leaf LeafMin A(0) 
derivation length: 170
production:

#Leaf(tts) : 
# for x in leafscales:
#   nproduce F(0.3) LeafMin(x) /(180) LeafMin(x) 
#   if x < 4:
#    for y in range(random.randint(1, 3)):
#      values = rand(10)
#      nproduce F(0.2)[-~l(values[y])] /(180) [+~l(values[y])]
# produce F(0.1)-(70) LeafMin(1)

Leaf(tts): 
 for x in leafscales:
    nproduce F(0.3) LeafMin(x) /(180) LeafMin(x) 
    randchoice = random.randint(1,3)
    for y in range(randchoice):
     values = rand(10)
     nproduce F(0.2)[-~l(values[y])] /(180)[+~l(values[y])]
 produce F(0.1) -(70) LeafMin(1)



LeafMin(tts) :
 produce  [+(70)F(tts*0.3)&(90) [\(120)&(90)~l(tts*0.5)] [\(60)&(270)~l(tts*0.5)] ^(90)F(tts*0.5)~l(tts*0.7)] 

A(n):
  produce F(0) Leaf(0)

interpretation:

endlsystem

LsystemWidget(animate=True, derivationLength=171, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe…

Working out how modules production and interpretation works

In [66]:
%%lpy -w 10 -a True

# Leafy axis
from openalea.plantgl.all import *
from math import * 

scaling = 5    # to dilate/contract the leaf
phi = 180      # Phyllotactic angle
h = 1          # height of an internode

module Leaf
module Fruit

Axiom: /(90)F(h) A(0)
derivation length: 10

production:

# try ^(-90)f(0.05)^(90) to translate the leaf on the periphery of the stem
A(n):
  produce [^(-45)Leaf(scaling)]F(h)[^(-45)Leaf(scaling)]F(h)[^(-45)Fruit(scaling)] F(h)/(phi)A(n+1) 

interpretation:
maximum depth: 2

# Organ definitions
Leaf(x) --> ~l
Fruit(x) --> F@O(0.5)
A(n) --> ;(2) @O(0.2)




endlsystem


LsystemWidget(animate=True, derivationLength=11, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe5…

Try integrating new leaf and a juvenile stage into Fred's model

In [67]:
%%lpy -w 10 -a True

from random import *
from random import randrange
from numpy.random import seed
from numpy.random import rand


module Apex
module DormantBud
module Inflorescence
module LatApex
module LeafMin
module Leaf

phyllochrone = 40
daytemp = None
juv = 0
seed(1)
leafscales = [0.5,1.5,1,0.5]

def StartEach():
  global daytemp 
  daytemp = 5
  
Axiom: Support ,(2) _(0.05) Apex(0, 1) Leaf LeafMin 

derivation length: 170

production:  


LeafMin(tts):
 produce [+(70)F(tts*0.3)&(90) [\(120)&(90)~l(tts*0.5)] [\(60)&(270)~l(tts*0.5)] ^(90)F(tts*0.5)~l(tts*0.7)] 

Leaf(tts): 
 for x in leafscales:
    nproduce F(0.3) LeafMin(x) /(180) LeafMin(x) 
    randchoice = randrange(0, 4)
    for y in range (0,3):
     values = rand(10)
     nproduce F(0.2)[-~l(values[y])] /(180)[+~l(values[y])]
    # continue
 produce F(0.1) -(70) LeafMin(1)

Apex(tts, state):
 global daytemp, phyllochrone, juv
 
 if tts >= phyllochrone:
   #nproduce [ &(60) Leaf(0) ]
   #nproduce [ &(60)~l(1)] 
   if juv>7:
      if random() < 0.5 and state <3 :
        nproduce [&(30) DormantBud(0, randint(20,30)) ]
        state +=1
      elif state == 3:
        #nproduce [ &(60) Leaf(0) ]
        #nproduce [ &(60)~l(1)] 
        nproduce [&(30) Inflorescence(0) ]
        state = 0
      else:
        nproduce [ &(60) Leaf(0) ]
        #nproduce [~l(1)]
        state +=1
   else:
       nproduce [ &(60) Leaf(0) ]
      #nproduce [ &(60)~l(1)]
   
   juv +=1
   tts -= phyllochrone
 
 nproduce /(144) Segment()
 produce Apex(tts+daytemp, state)

LatApex(tts, state):
  global daytemp, phyllochrone
  if tts >= phyllochrone:
    nproduce [ &(60) Leaf(0) ] 
    #nproduce [ &(60)~l(1)]
    if state == 3:
      nproduce [&(30) Inflorescence(0) ]
    else:
       if random() < 0.5:
          nproduce [&(30) DormantBud(0, randint(20,30)) ]
    state = state % 3
    state +=1
    tts -= phyllochrone
  nproduce /(144) Segment()
  produce LatApex(tts+daytemp, state)

DormantBud(tts, dtype):
  tts+=daytemp
  if tts >= dtype:
    nproduce LatApex(0,1)
  else:
    produce DormantBud(tts, dtype)

interpretation:

Segment() --> F(0.1)

Inflorescence(tts):
  for i in range(5):
   nproduce @Tp(0,0,-1) @Ts(0.2) _(0.02) F(0.2)[+(90)F(0.2) ,(3) @O(0.1)][-(90)F(0.2) ,(3) @O(0.1)]

endlsystem

LsystemWidget(animate=True, derivationLength=171, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe…

Try to flip leaf from production to interpretation

In [68]:
%%lpy -w 10 -a True

from random import *
from random import randrange
from numpy.random import seed
from numpy.random import rand


module Apex
module DormantBud
module Inflorescence
module LatApex
module LeafMin
module Leaf

phyllochrone = 40
daytemp = None
juv = 0
seed(1)
leafscales = [0.5,1.5,1,0.5]

def StartEach():
  global daytemp 
  daytemp = 5
  
Axiom: Support ,(2) _(0.05) Apex(0, 1) Leaf LeafMin 

derivation length: 170

production:  

Apex(tts, state):
 global daytemp, phyllochrone, juv
 
 if tts >= phyllochrone:
   #nproduce [ &(60) Leaf(tts) ]
   #nproduce [ &(60)~l(1)] 
   if juv>7:
      if random() < 0.5 and state <3 :
        nproduce [&(30) DormantBud(0, randint(20,30)) ]
        state +=1
      elif state == 3:
        #nproduce [ &(60) Leaf(tts) ]
        #nproduce [ &(60)~l(1)] 
        nproduce [&(30) Inflorescence(0) ]
        state = 0
      else:
        nproduce [ &(60) Leaf(0) ]
        #nproduce [~l(1)]
        state +=1
   else:
       nproduce [ &(60) Leaf(0) ]
      #nproduce [ &(60)~l(1)]
   
   juv +=1
   tts -= phyllochrone
 
 nproduce /(144) Segment()
 produce Apex(tts+daytemp, state)

LatApex(tts, state):
  global daytemp, phyllochrone
  if tts >= phyllochrone:
    nproduce [ &(60) Leaf(tts) ] 
    #nproduce [ &(60)~l(1)]
    if state == 3:
      nproduce [&(30) Inflorescence(0) ]
    else:
       if random() < 0.5:
          nproduce [&(30) DormantBud(0, randint(20,30)) ]
    state = state % 3
    state +=1
    tts -= phyllochrone
  nproduce /(144) Segment()
  produce LatApex(tts+daytemp, state)

DormantBud(tts, dtype):
  tts+=daytemp
  if tts >= dtype:
    nproduce LatApex(0,1)
  else:
    produce DormantBud(tts, dtype)

interpretation:

Segment() --> F(0.1)

Inflorescence(tts):
  for i in range(5):
   nproduce @Tp(0,0,-1) @Ts(0.2) _(0.02) F(0.2)[+(90)F(0.2) ,(3) @O(0.1)][-(90)F(0.2) ,(3) @O(0.1)]

#LeafMin(tts):
# produce [+(70)F(tts*0.3)&(90) [\(120)&(90)~l(tts*0.5)] [\(60)&(270)~l(tts*0.5)] ^(90)F(tts*0.5)~l(tts*0.7)] 

Leaf(tts):
 produce F(0.3) LeafMin() /(180) LeafMin()

LeafMin :
 produce [+(70)F(1*0.3)&(90) [\(120)&(90)~l(1*0.5)] [\(60)&(270)~l(1*0.5)] ^(90)F(1*0.5)~l(1*0.7)]

#Leaf(tts): 
# for x in leafscales:
#    nproduce F(0.3) LeafMin(x) /(180) LeafMin(x) 
#    randchoice = randrange(0, 4)
#    for y in range (0,3):
#     values = rand(10)
#     nproduce F(0.2)[-~l(values[y])] /(180)[+~l(values[y])]
#    # continue
# produce F(0.1) -(70) LeafMin(tts)




endlsystem

LsystemWidget(animate=True, derivationLength=171, is_magic=True, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe…